In [2]:
import tensorflow as tf
import numpy as np
import math
import gzip
import json
import datetime
from tqdm import tqdm

In [3]:
def reset_tf():
    global sess
    sess.close()
    tf.reset_default_graph()
    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [4]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [5]:
class HyperParameters():
    # maximum number of symbols in an input sequence
    max_sequence_length = 50
    
    # number of symbols in vocabulary
    # (symbols are expected to be in range(vocab_size))
    vocab_size = 10000

    # number of dimensions in input embeddings
    embedding_size = 128
    
    # number of sequences per batch
    batch_size = 64
    
    # number of target classes
    num_target_classes = 2

hp = HyperParameters()

In [19]:
dataset_filenames = ["../data/simplewiki/simplewiki-20171103.entity_recognition.tfrecords"]

def parse_example(example_proto):
    features = {
        'inputs': tf.FixedLenFeature((50), tf.int64),
        'word_endings': tf.FixedLenFeature((50), tf.int64),
        'targets': tf.FixedLenFeature((50), tf.int64)
    }
    parsed = tf.parse_single_example(example_proto, features)
    return parsed['inputs'], parsed['targets']

dataset = tf.data.TFRecordDataset(dataset_filenames)
dataset = dataset.map(parse_example)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

proto = sess.run(iterator.get_next())

InvalidArgumentError: Name: <unknown>, Key: word_endings, Index: 0.  Number of int64 values != expected.  Values size: 16 but output shape: [50]
	 [[Node: ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=3, Nsparse=0, Tdense=[DT_INT64, DT_INT64, DT_INT64], dense_shapes=[[50], [50], [50]], sparse_types=[]](ParseSingleExample/ExpandDims, ParseSingleExample/ParseExample/ParseExample/names, ParseSingleExample/ParseExample/ParseExample/dense_keys_0, ParseSingleExample/ParseExample/ParseExample/dense_keys_1, ParseSingleExample/ParseExample/ParseExample/dense_keys_2, ParseSingleExample/ParseExample/Const, ParseSingleExample/ParseExample/Const_1, ParseSingleExample/ParseExample/Const_2)]]
	 [[Node: IteratorGetNext_11 = IteratorGetNext[output_shapes=[[50], [50]], output_types=[DT_INT64, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_5)]]

Caused by op 'ParseSingleExample/ParseExample/ParseExample', defined at:
  File "/home/achang/anaconda3/

In [14]:
tf.parse_single_example(proto, {
    'inputs': tf.FixedLenFeature((), tf.int64),
    'word_endings': tf.VarLenFeature(tf.int64),
    'targets': tf.VarLenFeature(tf.int64)
})

{'inputs': <tf.Tensor 'ParseSingleExample_1/Squeeze_inputs:0' shape=() dtype=int64>,
 'targets': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f72dc4bdcc0>,
 'word_endings': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f72df4f50b8>}

In [12]:
input_sequences_dataset = []
input_lengths_dataset = []
input_word_endings_dataset = []
target_sequences_dataset = []

for sentence in tqdm(dataset):
    inputs = sentence['inputs']
    input_length = len(inputs)
    word_endings = sentence['word_endings']
    targets = sentence['targets']
    
    if input_length > hp.max_sequence_length:
        continue
    
    inputs = inputs + [0] * (hp.max_sequence_length - input_length)
    word_endings = word_endings + [0] * (hp.max_sequence_length - input_length)
    targets = targets + [0] * (hp.max_sequence_length - input_length)
    
    input_sequences_dataset.append(inputs)
    input_lengths_dataset.append(input_length)
    input_word_endings_dataset.append(word_endings)
    target_sequences_dataset.append(targets)

input_sequences_dataset = tf.convert_to_tensor(input_sequences_dataset)
input_lengths_dataset = tf.convert_to_tensor(input_lengths_dataset)
input_word_endings_dataset = tf.convert_to_tensor(input_word_endings_dataset)
target_sequences_dataset = tf.convert_to_tensor(target_sequences_dataset)

100%|██████████| 1095155/1095155 [00:08<00:00, 130964.65it/s]


In [15]:
train_imgs = tf.constant(['train/img1.png', 'train/img2.png',
                          'train/img3.png', 'train/img4.png',
                          'train/img5.png', 'train/img6.png'])
train_labels = tf.constant([0, 0, 0, 1, 1, 1])
train_data = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))

In [8]:
reset_tf()

# Placeholders
# ------------

# sequences of input symbols
input_sequences = tf.placeholder(tf.int32, 
                                 shape = (hp.batch_size, hp.max_sequence_length))
# length of each sequence
input_lengths = tf.placeholder(tf.int32, 
                               shape = (hp.batch_size))
# sequences of word ending markers
input_word_endings = tf.placeholder(tf.int32, 
                                    shape = (hp.batch_size, hp.max_sequence_length))
# sequences of target symbols
target_sequences = tf.placeholder(tf.int32, 
                                  shape = (hp.batch_size, hp.max_sequence_length))

# sequences of input positions (not a placeholder)
input_positions = tf.range(hp.max_sequence_length, dtype=tf.int32)
input_positions = tf.tile(input_positions, [hp.batch_size])
input_positions = tf.reshape(input_positions, 
                             (hp.batch_size, hp.max_sequence_length), 
                             name = 'input_positions')

# Embeddings
# ----------

# sequences of input embeddings w/ shape:
#   (hp.batch_size, hp.max_sequence_length, hp.embedding_size)
input_sequence_embeddings = tf.get_variable('input_sequence_embeddings', 
                                            (hp.vocab_size, hp.embedding_size))
input_sequences_embedded = tf.nn.embedding_lookup(input_sequence_embeddings, 
                                                  input_sequences,
                                                  name = 'input_sequences_embedded')

# sequences of input position embeddings w/ shape:
#   (hp.batch_size, hp.max_sequence_length, hp.embedding_size)
input_position_embeddings = tf.get_variable('input_position_embeddings', 
                                            (hp.max_sequence_length, hp.embedding_size))
input_positions_embedded = tf.nn.embedding_lookup(input_position_embeddings, input_positions)

# sequences of word ending embeddings w/ shape:
#   (hp.batch_size, hp.max_sequence_length, hp.embedding_size)
input_word_ending_embeddings = tf.get_variable('input_word_ending_embeddings',
                                               (2, hp.embedding_size))
input_word_endings_embedded = tf.nn.embedding_lookup(input_word_ending_embeddings, 
                                                     input_word_endings,
                                                     name = 'input_word_endings_embedded')

# Sequence mask
# -------------

sequence_mask = tf.sequence_mask(input_lengths,
                                 hp.max_sequence_length,
                                 dtype = tf.float32)
# expand dimensions to support broadcasting
expanded_sequence_mask = tf.expand_dims(sequence_mask, 
                                        2, 
                                        name = 'sequence')

input_combined_embedded = tf.add_n([input_sequences_embedded, 
                                    input_positions_embedded, 
                                    input_word_endings_embedded])
# TODO: is this necessary?
input_combined_embedded = tf.multiply(input_combined_embedded,
                                      expanded_sequence_mask,
                                      name = 'input_combined_embedded')

# Attention
# ---------

def attention_layer(A):
    A_T = tf.transpose(A, perm=[0, 2, 1])
    scaled_logits = tf.matmul(A, A_T) / tf.sqrt(tf.cast(tf.shape(A)[-1], tf.float32))
    return tf.matmul(tf.nn.softmax(scaled_logits), A)

# TODO: layer normalization (possibly with masking)
input_attention_layer_1 = input_combined_embedded + attention_layer(input_combined_embedded)

# Feed-forward
# ------------

def feed_forward_layer(A, num_units, scope='feed_forward', reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        A = tf.layers.dense(A, num_units, activation=tf.nn.relu)
        return tf.layers.dense(A, num_units)

# TODO: layer normalization (possibly with masking)
input_feed_forward_layer_1 = input_attention_layer_1 + feed_forward_layer(input_attention_layer_1, hp.embedding_size)

# TODO: how important is this masking?
input_feed_forward_layer_1 *= expanded_sequence_mask

# Softmax
# -------

output_logits = tf.layers.dense(input_feed_forward_layer_1, hp.num_target_classes)

# Loss
# ----

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=target_sequences,
                                                        logits=output_logits)
losses *= sequence_mask

total_loss = tf.reduce_sum(losses)
mean_loss  = total_loss / tf.cast(tf.reduce_sum(input_lengths), tf.float32)

# Training
# --------

optimizer = tf.train.AdamOptimizer(1e-3)

# gradient clipping
gradients, variables = zip(*optimizer.compute_gradients(mean_loss))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

train_op = optimizer.apply_gradients(zip(gradients, variables))

In [9]:
sess.run(tf.global_variables_initializer())

batch_indices = np.arange(len(input_sequences_dataset))

# for predictable rng
np.random.seed(0)

In [94]:
curr_input_sequences = np.zeros((hp.batch_size, hp.max_sequence_length), dtype=np.int32)
curr_input_lengths = np.zeros(hp.batch_size, dtype=np.int32)
curr_input_word_endings = np.zeros((hp.batch_size, hp.max_sequence_length), dtype=np.int32)
curr_target_sequences = np.zeros((hp.batch_size, hp.max_sequence_length), dtype=np.int32)

total_input_length = np.sum(input_lengths_dataset)
num_batches = (len(batch_indices) + hp.batch_size - 1) // hp.batch_size

for epoch in range(20):
    np.random.shuffle(batch_indices)

    epoch_loss = 0
    
    for i in range(num_batches):
        if i % 1000 == 0:
            print('epoch %d: batch %d/%d' % (epoch, i, num_batches))
        
        curr_indices = batch_indices[i*hp.batch_size:(i+1)*hp.batch_size]

        curr_input_sequences.fill(0)
        curr_input_lengths.fill(0)
        curr_input_word_endings.fill(0)
        curr_target_sequences.fill(0)

        x = input_sequences_dataset[curr_indices]
        curr_input_sequences[:x.shape[0]] = x

        x = input_lengths_dataset[curr_indices]
        curr_input_lengths[:x.shape[0]] = x

        x = input_word_endings_dataset[curr_indices]
        curr_input_word_endings[:x.shape[0]] = x

        x = target_sequences_dataset[curr_indices]
        curr_target_sequences[:x.shape[0]] = x

        feed_dict = {
            input_sequences: curr_input_sequences,
            input_lengths: curr_input_lengths,
            input_word_endings: curr_input_word_endings,
            target_sequences: curr_target_sequences
        }

        curr_loss, _ = sess.run((total_loss, train_op), feed_dict = feed_dict)
        epoch_loss += curr_loss
    
    epoch_loss /= total_input_length
    print('epoch %d: loss %g' % (epoch, epoch_loss))

epoch 0: batch 0/16517
epoch 0: batch 1000/16517
epoch 0: batch 2000/16517
epoch 0: batch 3000/16517
epoch 0: batch 4000/16517
epoch 0: batch 5000/16517
epoch 0: batch 6000/16517
epoch 0: batch 7000/16517
epoch 0: batch 8000/16517
epoch 0: batch 9000/16517
epoch 0: batch 10000/16517
epoch 0: batch 11000/16517
epoch 0: batch 12000/16517
epoch 0: batch 13000/16517
epoch 0: batch 14000/16517
epoch 0: batch 15000/16517
epoch 0: batch 16000/16517
epoch 0: loss 0.201803
epoch 1: batch 0/16517
epoch 1: batch 1000/16517
epoch 1: batch 2000/16517
epoch 1: batch 3000/16517
epoch 1: batch 4000/16517
epoch 1: batch 5000/16517
epoch 1: batch 6000/16517
epoch 1: batch 7000/16517
epoch 1: batch 8000/16517
epoch 1: batch 9000/16517
epoch 1: batch 10000/16517
epoch 1: batch 11000/16517
epoch 1: batch 12000/16517
epoch 1: batch 13000/16517
epoch 1: batch 14000/16517
epoch 1: batch 15000/16517
epoch 1: batch 16000/16517
epoch 1: loss 0.194863
epoch 2: batch 0/16517
epoch 2: batch 1000/16517
epoch 2: batc

KeyboardInterrupt: 